In [9]:
import requests
from  bs4 import BeautifulSoup as bs
import csv # для сохранениея результатов

headers = {'accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/'
          }
url='https://bryansk.hh.ru/search/vacancy?area=1&clusters=true&search_period=1&text=TeamLead&page=0'
hh_parse(url, headers)#запускаем функцию - не забудь поменять имя парсера!!!


def hh_parse(url, headers):
    jobs=[]
    session = requests.Session() #это эмуляция работы одного посетителя сайта в рамках одной сессии
    request = session.get(url, headers=headers)
    if request.status_code == 200: # парсеры включаются в работу после получения 200 кода ответа от сервера
        #start = time.time() #засекаем время работы парсера
        #print('OK')
        soup = bs(request.content, 'html.parser') # вариант с простым парсером
        divs=soup.find_all('div', attrs={'data-qa': 'vacancy-serp__vacancy'})
        #divs=soup.find_all('div', attrs={'class': 'vacancy-serp'})
        for div in divs:
            title = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'}).text #чтобы не ссылки, а названия вакансий выдал нам
            hrefs = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'})['href']
            company = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-employer'}).text
            text1 = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy_snippet_responsibility'}).text
            text2 = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy_snippet_requirement'}).text
            content = text1+' '+text2
            #print(title)#менять принт под каждый итер цикла для теста отработки
            jobs.append({
                'title': title,
                'href': hrefs,
                'company': company,
                'content': content 
            })
        print (str('Собрано вакансий: ') + str(len(jobs)))
    else:
        print('ERROR')

Собрано вакансий: 19


In [ ]:
'''См. часть 2 для записи результатов в файл'''
'''Во второй части надо установить через пип новую библу - lxml (ей пользовался Молчанов)'''

In [14]:
import requests
from  bs4 import BeautifulSoup as bs
import csv # для сохранениея результатов

headers = {'accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/'
          }
url='https://bryansk.hh.ru/search/vacancy?area=1&clusters=true&search_period=1&text=TeamLead&page=0'

jobs = wtf_hh(url, headers) #запускаем функцию - не забудь поменять имя парсера!!!
files_writer(jobs)# запуск функции записи в файл

ERROR or Done200


In [13]:
# Здесь уже парсим не только первую страницу запроса, а еще и пробежимся по страницам пагинации
def wtf_hh(url, headers):
    session = requests.Session() #это эмуляция работы одного посетителя сайта в рамках одной сессии
    r = session.get(url, headers=headers)
    jobs=[]
    urls=[] #список куда бум складывать урлы пагинации
    urls.append(url)#сразу добавим первую страницу, на тот случай, если она единственная (т.е. нет пагинации)
    if r.status_code == 200:# парсеры включаются в работу после получения 200 кода ответа от сервера
        #print('OK')
        soup = bs(r.content, 'lxml')  # вариант с продвинутым парсером
        try:
            pagination = soup.find_all('a', attrs={'data-qa': 'pager-page'})#говорим парсеру обнаружить все теги со ссылками на старницы пагинаци
            count = int(pagination[-1].text) #счетчик пагинации
            for i in range(count):
                url = f'https://bryansk.hh.ru/search/vacancy?area=1&clusters=true&search_period=1&text=TeamLead&page={i}' #итератор перебора урлов в пагинации
                if url not in urls:
                    urls.append(url)   
        except:
            pass # если не найдет пагинацию, то просто пойдет дальше
    for url in urls: # а это уже директива для перебора самих урлов пагинации в  списке
        r = session.get(url, headers=headers)
        soup = bs(r.content, 'lxml')    
        divs=soup.find_all('div', attrs={'data-qa': 'vacancy-serp__vacancy'})
        for div in divs:
            try: #чтобы не выбивало, если какой-то тег не обнаружен
                
                title = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'}).text #чтобы не ссылки, а названия вакансий выдал нам
                hrefs = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'})['href']
                company = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-employer'}).text
                text1 = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy_snippet_responsibility'}).text
                text2 = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy_snippet_requirement'}).text
                content = text1+' '+text2
                jobs.append({
                    'title': title,
                    'href': hrefs,
                    'company': company,
                    'content': content 
                })
            except:
                pass
            #for i in pagination:
                #print(pagination[-1].text) #происм вывести номер последней страницы, которая будет парсится
            #print(len(jobs))
    else:
        print('ERROR or Done'+ str(r.status_code))
    return (jobs) #если не сделать рутурн, то список jobs так и останется пустым!   

        
'''А теперь скинем все в csv'''
def files_writer(jobs):
    with open('python_hh_vacansies.csv', 'w', encoding='utf-8') as  file:
        a_pen = csv.writer(file)
        a_pen.writerow(('Название вакансии', 'URL', 'Название компании', 'Описание вакансии'))
        for job in  jobs:
            a_pen.writerow((job['title'],  job['href'], job['company'], job['content']))

In [50]:
import pandas as pd
data = jobs
df = pd.DataFrame(data)
df.columns = ['Название вакансии', 'URL', 'Название компании', 'Описание вакансии']
df.head()


,Название вакансии,URL,Название компании,Описание вакансии
0,TeamLead Android,https://bryansk.hh.ru/vacancy/35851629?query=T...,ООО Криптериум,Руководство отделом. Планирования и решения те...
1,Digital Content Media Team Lead,https://bryansk.hh.ru/vacancy/35792850?query=T...,FOREO Ltd.,Manage content advertising activities for the ...
2,"Team Lead (PHP, JS)",https://bryansk.hh.ru/vacancy/34941067?query=T...,BeeJee,"Экспертный уровень PHP-разработки (Yii, CI, K..."
3,Team lead Python developer,https://bryansk.hh.ru/vacancy/35846186?query=T...,BestDoctor,
4,User Acquisition Team Lead,https://bryansk.hh.ru/vacancy/35696020?query=T...,ООО G5 Entertainment AB,Организация работы UAM-ов (включая наем и обуч...


In [52]:
#Экспортнем все это дело в Эксель!
writer = pd.ExcelWriter('HH.xlsx')
df.to_excel(writer, 'TTLs')
writer.save()